# Imports

In [1]:
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel
import unicodedata

# linea que arregla algunos errores de loadeo de datasets
# pip install --upgrade datasets

/home/fabro/Documents/uba/aprendizaje_automatico/Puntualizacion-Capitalizacion/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento

In [2]:
linux = True
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

usando: cuda


# Ejericio b)

## Busqueda de fuentes

### Fuente 1: Conjunto de preguntas en espa;ol

In [3]:
questions, question_for_mixture = get_questions()

Se descargaron 5000 preguntas en Español.


### Fuente 2: Dataset provisto para Notebook 10

In [4]:
oraciones_rnn = get_notebook_dataset()

Se descargaron 997 oraciones en Español (del dataset del notebook 10).


### Fuente 3: Dataset sintetico generado con Gemini

In [5]:
oraciones_sinteticas = get_gemini_dataset()

Hay 1413 oraciones sintéticas.


### Fuente 4: Articulos de Wikipedia

In [6]:
frases_wikipedia = get_wikipedia_dataset()

['Argentina, oficialmente República Argentina,[a]\u200b es un país soberano de América del Sur, ubicado en el extremo sur y sudeste de ese subcontinente.', 'Adopta la forma de gobierno republicana, democrática, representativa y federal.', 'Poseen Carta Magna, bandera y fuerzas de seguridad propias, el dominio de los recursos naturales circunscriptos en su territorio y delegan los poderes exclusivos al Gobierno Federal.', 'Hasta mediados del siglo XX, fue una de las economías más prósperas del mundo.', 'No obstante, es la segunda economía más importante de Sudamérica —detrás de Brasil— y la 24.º más grande del mundo por PIB nominal.']


### Fuente 5: Subtitulos de peliculas

In [7]:
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()

✅ Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
✅ Frases extraídas y guardadas. Total: 947
['Mamá Cora.', '¡Mamá Cor... Mamá Cora!', '¡Acabala, carajo! ¡Cerrá el pico de una vez! Estás hablando de mi madre, ¿no? Y vos, ¿por qué no le buscás un lugarcito? Al fin y al cabo sos tan hijo de ella como yo. Además, tu mujer tiene mucha más paciencia que la mía.', 'Es un sueño.', 'Mejor, ¿quién te necesita?', 'No...', 'Abrígate. Mirá que cuando vuelvas va a refrescar.', 'Angelito, no llores más, mi amor te vas a hacer mal.', 'Por favor. No empecés otra vez con esa cantinela...', '¿No sabés que duerme? Atendélo vos.']
✅ Se extrajeron 1000 frases de Relatos Salvajes.


### Fuente 6 (beta): Mixture de oraciones

In [8]:
cant_oraciones = len(oraciones_sinteticas)
question_for_mixture = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", q).strip()) for q in question_for_mixture]
oraciones_sinteticas = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", a).strip()) for a in oraciones_sinteticas]

tanda_1 = question_for_mixture[:cant_oraciones]
question_affirmation = [f"{q} {a}" for q, a in zip(tanda_1, oraciones_sinteticas)]

tanda_2 = question_for_mixture[cant_oraciones:2*cant_oraciones]
affirmation_question = [f"{a} {q}" for q, a in zip(tanda_2, oraciones_sinteticas)]

tanda_3 = question_for_mixture[2*cant_oraciones:3*cant_oraciones]
tanda_3_shuffled = random.sample(tanda_3, len(tanda_3))
question_question = [f"{q} {p}" for q, p in zip(tanda_3, tanda_3_shuffled)]

mixtures = question_affirmation + affirmation_question + question_question

random.sample(mixtures, 5)


['¿De qué color es la circunferencia de la bandera nipona? ¿De qué corona es archivo oficial el Archivo General de Simancas?',
 'Mi profesor de matemáticas, el señor López, es muy claro. ¿Qué mando de Wii es opcional para la mayoría de eventos del videojuego?',
 '¿Por dónde viajó Obama en este periplo internacional? ¿Cuándo le encargaron la placa a Xavier?',
 'El Walkman fue un producto icónico de Sony. ¿Dónde ha mejorado sensiblemente la seguridad ciudadana?',
 '¿Qué significan las siglas MRE? ¿Dónde debe jugar un futbolista?']

## Juntamos las fuentes

In [9]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones Compuestas:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones Compuestas: 4239
Algunas oraciones aleatorias:


['¿Que tal?',
 '“Oye, Cohren, ¿Harris aún tiene a Bola de Nieve?”, preguntó Nemic mientras se ponían cómodos.',
 'C., concluyendo con la conquista de los españoles.',
 '¿Cuál es la población de la zona si la comparamos con la que había a mediados del siglo pasado?',
 'A diferencia de la Primera Guerra Mundial, la rendición (tanto la japonesa como la alemana) se produjo por derrota incondicional, sin pasar por ningún tipo de negociación.']

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

19231
1013


## Importamos el modelo

### Sin atencion

In [ ]:
from train.RNNBidirectional import PunctuationCapitalizationRNNBidirectional

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

bert_embeddings = bert_model.embeddings.word_embeddings
for param in bert_model.parameters():
    param.requires_grad = False

N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True


model = PunctuationCapitalizationRNNBidirectional(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

ckpt = torch.load("model_bidirec.pt", map_location=device)
# si guardaste state_dict puro
if isinstance(ckpt, dict) and "model_state_dict" not in ckpt:
    model.load_state_dict(ckpt)

# si guardaste un dict con más cosas (epoch, optim, etc.)
elif "model_state_dict" in ckpt:
    model.load_state_dict(ckpt["model_state_dict"])

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 180,944,905
Trainable parameters: 17,857,801


### Con atencion

In [14]:
from train.RNNBidirectionalAttention import PunctuationCapitalizationRNNBidirectionalAttention 

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

bert_embeddings = bert_model.embeddings.word_embeddings
for param in bert_model.parameters():
    param.requires_grad = False

N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True


model = PunctuationCapitalizationRNNBidirectionalAttention(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)



total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 180,683,785
Trainable parameters: 17,596,681


In [ ]:
punct_weights_tensor, cap_weights_tensor = compute_class_weights(
    dataloader_train,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS),
    device=device,
    beta=0.7
)

criterion_punct = nn.CrossEntropyLoss(ignore_index=-100, weight=punct_weights_tensor)
criterion_cap   = nn.CrossEntropyLoss(ignore_index=-100, weight=cap_weights_tensor)

"""
trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.rnn.parameters()) \
  + list(model.punct_classifier.parameters()) \
  + list(model.cap_classifier.parameters())
"""

trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.lstm1.parameters()) \
  + list(model.lstm2.parameters()) \
  + list(model.attention.parameters()) \
  + list(model.punct_classifier.parameters()) \
  + list(model.cap_classifier.parameters())

optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)

train(model, dataloader_train=dataloader_train, dataloader_test=dataloader_test,optimizer=optimizer, criterion_punct=criterion_punct, criterion_cap = criterion_cap, device=device, epochs=20)

AttributeError: 'PackedSequence' object has no attribute 'dim'

## Evaluacion

In [ ]:
evaluate(model, dataloader_test, device)

In [36]:
entrada = "quiero aprender a programar en python conocés algún curso"
print(f"{predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

Quiero aprender a programar en Python. Conocés algún curso.


## Export modelo

In [22]:
torch.save(model.state_dict(), "model_bidirec_more_data.pt")